# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [4]:
from bs4 import BeautifulSoup
import requests


# Standard headers to fetch a website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:2_000]


def fetch_website_links(url):
    """
    Return the links on the webiste at the given url
    I realize this is inefficient as we're parsing twice! This is to keep the code in the lab simple.
    Feel free to use a class and optimize it!
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    links = [link.get("href") for link in soup.find_all("a")]
    return [link for link in links if link]


In [6]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [7]:
# these is one shot prompt
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [ ]:
# baking the prompt
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwar

In [9]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"} # response format to parse json object, tell the model to respond in json format strictly, c
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [10]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/'},
  {'type': 'blog post',
   'url': 'https:// edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/'},
  {'type': 'product page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin profile', 'url': 'https://ww

In [11]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [12]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 8 relevant links


{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'partner page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [13]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


{'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'Status page', 'url': 'https://status.huggingface.co/'},
  {'type': 'API endpoints', 'url': 'https://endpoints.huggingface.co'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [14]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [15]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
deepseek-ai/DeepSeek-OCR
Updated
about 18 hours ago
•
623k
•
1.82k
PaddlePaddle/PaddleOCR-VL
Updated
1 day ago
•
14.8k
•
1.07k
tencent/HunyuanWorld-Mirror
Updated
about 13 hours ago
•
5.94k
•
331
krea/krea-realtime-video
Updated
5 days ago
•
1.04k
•
181
Qwen/Qwen3-VL-8B-Instruct
Updated
10 days ago
•
262k
•
321
Browse 1M+ models
Spaces
Running
515
515
veo3.1-fast
🐨
Generate videos from text or images
Running
15.4k
15.4k
DeepSite v3
🐳
Generate any application by Vibe Coding
Running
on
Zero
227
227
DeepSeek OCR Demo
🆘
An interactive de

In [16]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [18]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 6 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-OCR\nUpdated\nabout 18 hours ago\n•\n623k\n•\n1.82k\nPaddlePaddle/PaddleOCR-VL\nUpdated\n1 day ago\n•\n14.8k\n•\n1.07k\ntencent/HunyuanWorld-Mirror\nUpdated\nabout 13 hours ago\n•\n5.94k\n•\n332\nkrea/krea-realtime-video\nUpdated\n5 days ago\n•\n1.04k\n•\n181\nQwen/Qwen3-VL-8B-Instruct\nUpdated\n10 days ago\n•\n262k\n•\n321\nBrowse 1M+ models\nSpaces\nRunning\n515\n515\

In [19]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [20]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 15 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the premier AI community and collaboration platform dedicated to building the future of machine learning. It serves as the central hub where machine learning engineers, scientists, developers, and enthusiasts come together to create, share, and advance open-source AI models, datasets, and applications.

The platform supports all modalities including text, image, video, audio, and even 3D, fostering innovation across the entire spectrum of machine learning.

---

## Platform Highlights

- **1M+ Models Available**  
  Access an extensive and rapidly growing repository of machine learning models in various domains, updated frequently by a global community.

- **250k+ Datasets**  
  Discover and collaborate on a vast array of datasets, supporting diverse AI training and research needs.

- **400k+ Applications in Spaces**  
  Explore and deploy machine learning applications in "Spaces," where users can run and demo models seamlessly.

- **Open Source Stack**  
  Build faster and better using Hugging Face's open source tools, enabling accelerated development and experimentation.

- **Enterprise-Grade Solutions**  
  Hugging Face offers paid Compute and Enterprise plans with advanced features — security, dedicated support, and access controls — designed for teams and organizations to build AI at scale.

---

## Community & Collaboration

Hugging Face is more than a platform; it is a vibrant community powering ethical and open machine learning innovation. The company fosters an inclusive environment where knowledge sharing and creativity thrive.

- Collaborate on unlimited public projects  
- Share your ML portfolio with the world  
- Learn from and contribute to cutting-edge AI research  
- Engage with tens of thousands of developers, researchers, and organizations

---

## Company Culture

Hugging Face champions transparency, openness, and community-driven progress. Their culture emphasizes:

- Open-source ethos and collaboration  
- Ethical AI development for a responsible future  
- Empowerment of machine learning engineers through education and tools  
- Supportive environment encouraging innovation and diversity  

---

## Careers & Opportunities

Hugging Face is continuously growing and searching for passionate individuals who want to:

- Build and maintain scalable AI infrastructure  
- Contribute to open-source projects impacting millions worldwide  
- Innovate at the intersection of research and industry  
- Work with a globally distributed, diverse team dedicated to building the AI community of tomorrow  

If you are a machine learning engineer, researcher, developer, or product innovator, Hugging Face offers enriching career paths with opportunities to shape the future of AI.

---

## Customers & Use Cases

Hugging Face's platform supports a wide array of users:

- Independent developers building AI-powered applications  
- Researchers accelerating AI experiments with powerful tools  
- Enterprises integrating AI with secure, scalable infrastructure  
- Educators and students leveraging datasets and models for learning  

Top-performing models include OCR solutions, video generation, and instruction-following language models, demonstrating the platform's versatility and real-world impact.

---

## Get Started

Join the largest machine learning community and accelerate your AI projects today:

- Explore 1M+ models & 250k+ datasets  
- Host and share your own ML work  
- Access enterprise-grade compute solutions  
- Collaborate within a vibrant, ethical AI community  

**Sign up now** and be part of building the AI future with Hugging Face.

---

## Brand Assets

- **Logo Colors:** #FFD21E (primary yellow), #FF9D00 (secondary orange), #6B7280 (gray)  
- Various logo formats available (.svg, .png, .ai) for use in projects and publications.

---

Hugging Face — Empowering the next generation of machine learning engineers to build an open and ethical AI future.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [21]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True # 
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is a dynamic AI community and collaboration platform shaping the future of machine learning (ML). It serves as a central hub where machine learning engineers, data scientists, and AI enthusiasts can discover, share, and build the next generation of AI models, datasets, and applications. 

With over **1 million models**, **250,000 datasets**, and **400,000+ applications** publicly accessible, Hugging Face is the home of open-source ML development. It enables users to move faster, collaborate openly, and explore across various AI modalities such as text, image, video, audio, and even 3D.

---

## What Hugging Face Offers

- **Models Hub:** Browse and explore over a million pre-trained machine learning models, including trending ones like DeepSeek-OCR and PaddleOCR.

- **Datasets Library:** Access over 250,000 datasets perfect for training and evaluation of ML models, with up-to-date contributions from the community.

- **Spaces:** Deploy and share AI applications easily. Currently hosting over 500 live applications ranging from interactive demos to video generation tools.

- **Open Source Stack:** Accelerate your ML projects with Hugging Face's open source tools and libraries optimized for collaborative AI development.

- **Enterprise Solutions:** For teams and organizations, Hugging Face provides enterprise-grade security, access control, dedicated support, and paid compute resources to scale AI initiatives securely and efficiently.

---

## Company Culture

Hugging Face cultivates an open, collaborative, and ethical AI ecosystem. It empowers the global machine learning community to learn, share, and experiment transparently to build an inclusive and responsible AI future. The platform values:

- **Community Engagement:** Encouraging sharing and collaboration within a fast-growing and diverse AI community.

- **Open Source Ethos:** Driving innovation through openness and accessibility of AI technologies.

- **Multidisciplinary Exploration:** Supporting AI development across all modalities from text and vision to audio and 3D.

---

## Our Customers

Hugging Face serves a broad spectrum of users, including:

- Individual AI researchers and developers building and sharing models and datasets.
- Startups and enterprises leveraging the platform for scalable AI solutions with enterprise-grade security.
- Academics and educators using the tools and datasets for research and teaching.
- AI enthusiasts and the broader community interested in cutting-edge machine learning applications.

---

## Careers and Opportunities

Join Hugging Face to be part of an AI community that is building the future. The company invites passionate individuals who want to contribute to open-source AI and create impact at the forefront of machine learning innovation.

Openings span engineering, research, product development, community management, and enterprise support roles, focused on pushing the boundaries of AI collaboration and accessibility.

---

## Get Started with Hugging Face

- **Explore AI Apps** and find the perfect model for your project.
- **Host and share** your ML models, datasets, and applications for free.
- **Build your portfolio** and establish your presence in the AI community.
- **Accelerate development** with flexible paid compute and enterprise solutions for teams.

Sign up today at [huggingface.co](https://huggingface.co) and be part of the AI revolution!

---

## Brand Highlights

- Hugging Face’s signature colors include bright yellow (#FFD21E) and vibrant orange (#FF9D00), symbolizing warmth and innovation.
- The logo and brand assets represent a friendly, approachable community dedicated to collaborative AI progress.

---

Hugging Face — *The AI community building the future.*

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


# Welcome to Hugging Face – The AI Community Building the Future!

---

## Who Are We?  
Imagine a place where machine learning wizards, data sorcerers, and AI alchemists gather to share their spells — uh, models — datasets, and apps. That’s Hugging Face! We’re *the* platform where the AI community collaborates, creates, and sometimes even has a little fun while building the future.

Our motto? **"Keep it open. Keep it ethical. Keep it hugging."** 💛

---

## What’s Cooking in the AI Kitchen?

- **1 Million+ Models** — From image generators to language wizards, our treasure trove of open-source ML models grows faster than you can say "neural network."  
- **250,000+ Datasets** — Feeding AI brains with everything from chat prompts to persona profiles. Hungry for data? Dig in!  
- **400,000+ Applications & Spaces** — Launch apps, share your ML portfolio, or just show off cool demos that make your friends say, “Whoa, AI can do that?”  
- **Multimodal Madness** — Text, image, video, audio, even 3D...if AI had a Swiss Army knife, we’d be it.  

---

## Customers & Community  
Whether you’re a student trying to get your AI feet wet, a startup looking to scale your genius, or an enterprise aiming to deploy heavy-duty models in the real world, Hugging Face has your back.

With the fastest growing community of *machine learning enthusiasts* and the support of some seriously big names and organizations, here’s a place where:

- **Freelancers** can build a portfolio and get noticed.  
- **Researchers** can push boundaries openly and ethically.  
- **Businesses** can accelerate AI adoption with our paid Compute and Enterprise suites.  

Join 1.29k+ Spaces and thousands more running models that power everything from video generation to AI-powered image editing.

---

## Culture & Career – Geek Out with Us!  
We believe collaboration beats isolation every day. Our culture?

- Open source at heart ❤️  
- Ethical AI advocates  
- Casual tea-drinkers and serious problem solvers  
- Always learning, always sharing, always growing  

Want to build machine learning tools that millions will use? Hugging Face is where your skills meet endless possibilities. From ML engineers to community managers, our doors are wide open (virtual hugs included).

---

## Speed Up Your AI Journey  
No need to code in the dark alone or fight for GPU time — deploy models and apps with a few clicks on optimized inference endpoints, starting at just $0.60/hour for GPU!

Whether you want to host that killer new model or just tweak an existing one, we give you the tools and community support to **move faster, build smarter, and hug tighter**.

---

## Quick Hugging Face Facts  
- **Founded:** Around the corner from the future  
- **Colors:** Bright yellow (#FFD21E), orange (#FF9D00), and sleek gray (#6B7280) — because AI should be as vibrant as its ideas!  
- **Mascot:** Friendly face with a warm smile (because AIs could learn a thing or two about friendliness here)  

---

## Ready to Join the AI Hug Circle?  

Sign up, share your work, explore millions of models and datasets, and get your AI career (or project!) hugging new heights.

[Explore AI Apps](#) | [Browse 1M+ Models](#) | [Sign Up & Join The Fun](#)

---

*Hugging Face — where the future of AI isn’t just created; it’s hugged into existence.* 🤗✨

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>